In [1]:
# pip install pydantic

In [2]:
# pip install boto3

In [3]:
# pip install gspread

In [4]:
# pip install openai

In [1]:
import sys
import os


repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(repo_root)
print("Added to PYTHONPATH:", repo_root)

Added to PYTHONPATH: /home/jovyan/work


In [2]:
import json
from typing import Any, Dict

import boto3
import yaml

from core.task_processor import TaskProcessor
from core.process_registry import ProcessRegistry
from core.task_publisher import TaskPublisher

from core.processes.client_bank_statement import ClientBankStatementProcess

In [3]:
tenant_id = "dev"

with open(f"../config/tenants/{tenant_id}.yaml", "r") as f:
    tenant_config: Dict[str, Any] = yaml.safe_load(f)

dynamodb_config = tenant_config.get("dynamodb", {})
dynamodb = boto3.resource(
    "dynamodb",
    region_name=dynamodb_config.get("region"),
)
messages_table = dynamodb.Table(dynamodb_config.get("messages_table"))
processes_table = dynamodb.Table(dynamodb_config.get("processes_table"))
contacts_table = dynamodb.Table(dynamodb_config.get("contacts_table"))
tasks_table = dynamodb.Table(dynamodb_config.get("tasks_table"))

sqs_config = tenant_config.get("sqs", {})
sqs = boto3.client("sqs", region_name=sqs_config.get("region"))
queue_url = sqs_config.get("tasks_url")

In [4]:
task_publisher = TaskPublisher(sqs, queue_url)

processor = TaskProcessor(
    tenant_config=tenant_config,
    messages_table=messages_table,
    processes_table=processes_table,
    contacts_table=contacts_table,
    tasks_table=tasks_table,
    task_publisher=task_publisher,
)

In [6]:
body = {
    "task_id": "9e9e0207-bfde-4183-831a-d639b341d99d",
    "task_type": "VALIDATE_INCOMING_WHATSAPP_DOCUMENT",
    "agent_type": "ACCOUNTING_ASSISTANT",
    "process_type": "PROCESS_INCOMING_WHATSAPP_DOCUMENT",
    "context_key": {
        "identity": "whatsapp:525571969848",
        "msg_id": "wamid.HBgNNTIxNTU3MTk2OTg0OBUCABIYFDRBMjZFMjVFMkYzRjNCOTlGREI3AA=="
    },
    "payload": {
        "phone": "525571969848",
        "timestamp_epoch": 1766041439,
        "media_id": "1401087505057088"
    },
    "timestamp_iso": "2025-12-18T07:03:59",
    "timestamp_epoch": 1766041439
}
task = json.dumps(body)

In [7]:
processed, remaining = processor.process(task)

📤 Publishing task 5879da4a-3ddf-4e10-b4bc-31f59ba0c77c → EXTRACT DATA delay=0s
